In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import re
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_profiling
import plotly.offline as po
import plotly.graph_objs as go
%matplotlib inline


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

<ipython-input-1-470b38b36f52>:19: FutureWarning:

Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.



# Take a look at feature_data 

In [2]:
feature_data =  pd.read_csv('features_data.csv')
feature_data.shape
feature_data.head()

(5584, 9)

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91


# Take a look at equity_data 

In [3]:
equity_data = pd.read_csv('equity_value_data.csv')
equity_data.shape
equity_data.head()

(1119158, 3)

,timestamp,close_equity,user_id
0,2016-11-16T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
1,2016-11-17T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
2,2016-11-18T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
3,2016-11-21T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03
4,2016-11-22T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03


In [4]:
# pandas_profiling.ProfileReport(equity_data)

In [5]:
# convert to datetime to facilitate identifying gap in using the app
equity_data['date'] = pd.to_datetime(equity_data['timestamp']).dt.date
equity_data['date_shift'] = equity_data.groupby('user_id')['date'].shift()
equity_data['day_lag'] = (equity_data['date'] - equity_data['date_shift']).dt.days

equity_data.head()

,timestamp,close_equity,user_id,date,date_shift,day_lag
0,2016-11-16T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-16,NaN,NaN
1,2016-11-17T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-17,2016-11-16,1.0
2,2016-11-18T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-18,2016-11-17,1.0
3,2016-11-21T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-21,2016-11-18,3.0
4,2016-11-22T00:00:00Z,48.16,bcef4fa9b0bdf22bcf7deae708decf03,2016-11-22,2016-11-21,1.0


In [6]:
# get users have gap greater than 28 days 
equity_data[equity_data['day_lag']>=28].shape

(324, 6)

In [7]:
# get the churned dataframe
churned_df = equity_data[equity_data['day_lag']>=28].drop_duplicates('user_id', keep = 'last')
churned_df['churned'] = True
churned_df.shape
churned_df.head()

(279, 7)

,timestamp,close_equity,user_id,date,date_shift,day_lag,churned
7887,2017-05-15T00:00:00Z,4945.22,270cda53a026bcf6c2b98492b23c1b99,2017-05-15,2017-03-03,73.0,True
8104,2017-07-27T00:00:00Z,20.15,319c069e77187c7e7e7c027eb00fd941,2017-07-27,2017-03-23,126.0,True
8274,2017-07-12T00:00:00Z,201.80,ec84f134c5b27c4f5702803e98eb3f40,2017-07-12,2017-06-06,36.0,True
10097,2016-12-13T00:00:00Z,200.00,6dbedf7f2972b4f6ea60a15d48cb292c,2016-12-13,2016-10-14,60.0,True
11254,2017-05-19T00:00:00Z,11.70,91837d41270b81b267fc205c3e03ee9b,2017-05-19,2017-03-31,49.0,True


# Question #1: What percentage of users have churned in the data provided
Churn rate = 279/5584 = 0.04996 ~= 5% 

In [8]:
# get churn column back to the feature dataset and set it as boolean
feature_data = pd.merge(feature_data, churned_df[['user_id','churned']], how='left', on='user_id')
feature_data['churned'].fillna(False, inplace=True)
feature_data['churned'] = feature_data['churned'] * 1

In [464]:
feature_data.shape
feature_data.head()

(5584, 12)

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,churned,first_deposit_amount_natural_log,first_deposit_amount_scaled
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034,0,1.60206,-0.280231
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638,0,2.30103,-0.204693
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f,0,1.39794,-0.287313
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393,0,2.00000,-0.251904
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91,0,1.30103,-0.289673


In [465]:
# view churned and non-churned
feature_data['churned'].value_counts().values

array([5305,  279], dtype=int64)

In [40]:
# log transform first deposit amount to make highly skewed distributions less skewed
feature_data['first_deposit_amount_natural_log'] = np.where(feature_data['first_deposit_amount'] > 0.0000000001, 
                                                            np.log10(feature_data['first_deposit_amount'] ), -10)

D:\anaconda\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log10



In [46]:
# aslo applied standardscaler to removes the mean and scales the feature/variable to unit variance
feature_data['first_deposit_amount_scaled'] = feature_data['first_deposit_amount']

from sklearn.preprocessing import StandardScaler

standardScaler = StandardScaler()
columns_for_ft_scaling = ['first_deposit_amount_scaled']

feature_data[columns_for_ft_scaling] = standardScaler.fit_transform(feature_data[columns_for_ft_scaling])

In [273]:
feature_data.head()

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,user_id,churned,first_deposit_amount_natural_log,first_deposit_amount_scaled
0,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,33.129417,stock,40.0,med_time_horizon,895044c23edc821881e87da749c01034,0,1.60206,-0.280231
1,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,16.573517,stock,200.0,short_time_horizon,458b1d95441ced242949deefe8e4b638,0,2.30103,-0.204693
2,med_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,10.008367,stock,25.0,long_time_horizon,c7936f653d293479e034865db9bb932f,0,1.39794,-0.287313
3,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,1.031633,stock,100.0,short_time_horizon,b255d4bd6c9ba194d3a350b3e76c6393,0,2.00000,-0.251904
4,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,8.187250,stock,20.0,long_time_horizon,4a168225e89375b8de605cbc0977ae91,0,1.30103,-0.289673


In [225]:
# separate dependent variable and independent variable
y = feature_data['churned']
X = feature_data_new.drop(['churned', 'user_id'], axis=1)

In [226]:
# create train and test data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state= 50)

In [227]:
# remove first_deposit_amount_natural_log and first_deposit_amount_scaled in order for the below WOE process, 
# potentailly want to see the impact of both WOE version feature and tranformed feature in the model
X_train.head()
two_cols = X_train[['first_deposit_amount_natural_log','first_deposit_amount_scaled']]
X_train_nocol = X_train.drop(columns=['first_deposit_amount_natural_log','first_deposit_amount_scaled'])

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,first_deposit_amount_natural_log,first_deposit_amount_scaled
1159,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,19.498700,reit,1000.0,long_time_horizon,3.00000,0.172998
5404,high_risk_tolerance,limited_investment_exp,somewhat_important_liq_need,iOS,0.000000,stock,500.0,long_time_horizon,2.69897,-0.063059
579,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,14.072367,stock,500.0,short_time_horizon,2.69897,-0.063059
1302,med_risk_tolerance,no_investment_exp,very_important_liq_need,Android,8.261133,stock,1.0,short_time_horizon,0.00000,-0.298643
69,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,16.145000,stock,50.0,long_time_horizon,1.69897,-0.275510


In [228]:
X_test.head()
two_cols_test = X_test[['first_deposit_amount_natural_log', 'first_deposit_amount_scaled']]
X_test_nocol = X_test.drop(columns=['first_deposit_amount_natural_log', 'first_deposit_amount_scaled'])

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,first_deposit_amount_natural_log,first_deposit_amount_scaled
750,high_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,60.503183,stock,100.00,long_time_horizon,2.000000,-0.251904
650,high_risk_tolerance,limited_investment_exp,very_important_liq_need,iOS,38.129083,stock,0.26,short_time_horizon,-0.585027,-0.298993
1719,med_risk_tolerance,good_investment_exp,somewhat_important_liq_need,iOS,4.601550,stock,2000.00,short_time_horizon,3.301030,0.645112
159,med_risk_tolerance,limited_investment_exp,very_important_liq_need,Android,0.000000,etp,100.00,med_time_horizon,2.000000,-0.251904
3139,high_risk_tolerance,no_investment_exp,somewhat_important_liq_need,iOS,45.293850,adr,46.00,short_time_horizon,1.662758,-0.277398


In [230]:
# applied WOE as it helps to transform a continuous independent variable into a set of 
# groups or bins based on similarity of dependent variable
# distribution i.e. number of events and non-events.
from xverse.transformer import WOE

clf = WOE()
clf.fit(X_train_nocol, y_train)

D:\anaconda\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning:

divide by zero encountered in log



WOE(mono_custom_binning={'first_deposit_amount': array([    0.,    50.,   300., 50000.]),
                         'time_spent': array([0.00000000e+00, 5.73203333e+00, 2.36597167e+01, 8.78832945e+03])},
    woe_bins={'first_deposit_amount': {Interval(-0.001, 50.0, closed='right'): 0.4731469409338814,
                                       Interval(50.0, 300.0, closed='right'): -0.16785846859875064,
                                       Interval(300.0, 50000.0, closed='right'): -0.59...
                                 'med_risk_tolerance': 0.27647692119036293},
              'time_horizon': {'long_time_horizon': 0.012759776755320251,
                               'med_time_horizon': -0.22884021000411292,
                               'short_time_horizon': 0.0586328729113119},
              'time_spent': {Interval(-0.001, 5.732, closed='right'): -0.20729106860524574,
                             Interval(5.732, 23.66, closed='right'): 0.133894518881831,
                             I

In [232]:
clf.woe_df

,Variable_Name,Category,Count,Event,Non_Event,Event_Rate,Non_Event_Rate,Event_Distribution,Non_Event_Distribution,WOE,Information_Value
0,first_deposit_amount,"(-0.001, 50.0]",1324,106,1218,0.080060,0.919940,0.527363,0.328568,0.473147,0.189217
1,first_deposit_amount,"(50.0, 300.0]",1346,59,1287,0.043834,0.956166,0.293532,0.347181,-0.167858,0.189217
2,first_deposit_amount,"(300.0, 50000.0]",1238,36,1202,0.029079,0.970921,0.179104,0.324251,-0.593550,0.189217
3,instrument_type_first_traded,0,10,1,9,0.100000,0.900000,0.004975,0.002428,0.717449,0.019739
4,instrument_type_first_traded,adr,134,7,127,0.052239,0.947761,0.034826,0.034260,0.016396,0.019739
5,instrument_type_first_traded,cef,13,1,12,0.076923,0.923077,0.004975,0.003237,0.429767,0.019739
6,instrument_type_first_traded,etp,283,15,268,0.053004,0.946996,0.074627,0.072296,0.031737,0.019739
7,instrument_type_first_traded,lp,4,1,3,0.250000,0.750000,0.004975,0.000809,1.816061,0.019739
8,instrument_type_first_traded,mlp,40,2,38,0.050000,0.950000,0.009950,0.010251,-0.029766,0.019739
9,instrument_type_first_traded,reit,40,0,40,0.000000,1.000000,0.000000,0.010790,0.000000,0.019739


In [233]:
clf.iv_df

,Variable_Name,Information_Value
0,first_deposit_amount,0.189217
5,risk_tolerance,0.124436
7,time_spent,0.020129
1,instrument_type_first_traded,0.019739
2,investment_experience,0.018244
4,platform,0.015780
6,time_horizon,0.009736
3,liquidity_needs,0.000882


In [234]:
# transform WOE version of features back to train and test dataframe

X1 = clf.transform(X_train_nocol)
X2 = clf.transform(X_test_nocol)

In [236]:
# add back the first_deposit_amount_natural_log and first_deposit_amount_scaled columns removed above 

# X1 is the new training set 
X1_new = pd.concat([X1, two_cols], axis=1)
X1_new

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,first_deposit_amount_natural_log,first_deposit_amount_scaled
1159,-0.276759,0.007658,0.027015,-0.074314,0.133895,0.000000,-0.593550,0.012760,3.000000,0.172998
5404,-0.276759,0.007658,0.027015,-0.074314,-0.207291,-0.006862,-0.593550,0.012760,2.698970,-0.063059
579,0.276477,0.007658,-0.014207,0.036066,0.133895,-0.006862,-0.593550,0.058633,2.698970,-0.063059
1302,0.276477,0.127900,-0.014207,0.036066,0.133895,-0.006862,0.473147,0.058633,0.000000,-0.298643
69,-0.276759,0.007658,-0.014207,-0.074314,0.133895,-0.006862,0.473147,0.012760,1.698970,-0.275510
...,...,...,...,...,...,...,...,...,...,...
3330,-0.276759,0.127900,0.027015,-0.074314,0.133895,-0.006862,-0.167858,0.058633,2.000000,-0.251904
70,-0.276759,-0.262078,-0.014207,-0.074314,0.133895,-0.006862,-0.593550,0.012760,3.176091,0.409055
132,-0.276759,-0.262078,-0.014207,-0.074314,-0.207291,-0.006862,0.473147,-0.228840,1.698970,-0.275510
2014,-0.276759,0.007658,0.027015,-0.074314,0.045963,-0.006862,-0.167858,0.012760,2.000000,-0.251904


In [237]:
# X2 is the new testing set 
X2_new = pd.concat([X2, two_cols_test], axis=1)
X2_new

,risk_tolerance,investment_experience,liquidity_needs,platform,time_spent,instrument_type_first_traded,first_deposit_amount,time_horizon,first_deposit_amount_natural_log,first_deposit_amount_scaled
750,-0.276759,0.007658,-0.014207,0.036066,0.045963,-0.006862,-0.167858,0.012760,2.000000,-0.251904
650,-0.276759,0.007658,-0.014207,-0.074314,0.045963,-0.006862,0.473147,0.058633,-0.585027,-0.298993
1719,0.276477,-0.262078,0.027015,-0.074314,-0.207291,-0.006862,-0.593550,0.058633,3.301030,0.645112
159,0.276477,0.007658,-0.014207,0.036066,-0.207291,0.031737,-0.167858,-0.228840,2.000000,-0.251904
3139,-0.276759,0.127900,0.027015,-0.074314,0.045963,0.016396,0.473147,0.058633,1.662758,-0.277398
...,...,...,...,...,...,...,...,...,...,...
2273,-0.276759,-0.262078,-0.014207,0.036066,0.045963,-0.006862,-0.593550,0.012760,3.301030,0.645112
1683,-0.276759,0.127900,-0.014207,0.036066,0.133895,0.000000,0.473147,0.058633,1.000000,-0.294394
2845,0.276477,0.007658,-0.014207,0.337491,0.045963,-0.006862,-0.593550,0.012760,3.000000,0.172998
2155,0.276477,0.127900,-0.014207,-0.074314,0.133895,-0.029766,0.473147,0.058633,1.000000,-0.294394


In [80]:
import numpy as np
import statsmodels.api as sm

In [267]:
# remove columns that have high p value, potentially indicates that these features are not strong enough to suggest an effect exists in the population
# could also due to small sample size 
X1_new_1 = X1_new.drop(columns={'liquidity_needs', 'first_deposit_amount_natural_log','first_deposit_amount_scaled','investment_experience'})
X2_new_2 = X2_new.drop(columns={'liquidity_needs', 'first_deposit_amount_natural_log','first_deposit_amount_scaled','investment_experience'})

In [466]:
# fit logistic regression
logit_model = sm.Logit(y_train, X1_new_1, maxiter=1000)
result = logit_model.fit_regularized()
y_pred = result.predict(X2_new_2)

from sklearn.metrics import (confusion_matrix, accuracy_score)
from sklearn import metrics

# get AUC stat on testing set
fpr, tpr, thresholds = metrics.roc_curve(y_test, y_pred, pos_label=1)
print(metrics.auc(fpr, tpr))

Optimization terminated successfully    (Exit mode 0)
            Current function value: 0.6524223490912433
            Iterations: 50
            Function evaluations: 50
            Gradient evaluations: 50
0.6029853021404962


In [467]:
# get AUC stat on training set
y_pred1 = result.predict(X1_new_1)
fpr, tpr, thresholds = metrics.roc_curve(y_train, y_pred1, pos_label=1)
print(metrics.auc(fpr, tpr))

0.6468366288331743


In [468]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                churned   No. Observations:                 3908
Model:                          Logit   Df Residuals:                     3902
Method:                           MLE   Df Model:                            5
Date:                Thu, 25 Nov 2021   Pseudo R-squ.:                  -2.218
Time:                        14:40:28   Log-Likelihood:                -2549.7
converged:                       True   LL-Null:                       -792.20
Covariance Type:            nonrobust   LLR p-value:                     1.000
================================================================================================
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
risk_tolerance                   0.9957      0.103      9.661      0.000       0.794       1.198
platform                         1.0365      0.281      3.692      0.000       0.486       1.587
time_spent                       0.6870      0.232      2.958      0.003       0.232       1.142
instrument_type_first_traded    -0.7775      0.351     -2.215      0.027      -1.466      -0.089
first_deposit_amount             0.9307      0.076     12.206      0.000       0.781       1.080
time_horizon                     0.6945      0.328      2.120      0.034       0.053       1.336
================================================================================================
"""

In [410]:
# combine test and training set to calculate KS score

# first get all probability of churn predicted by the model
y_all = y_pred.append(y_pred1)
prob_combined = y_all.reset_index()
prob_combined.rename(columns={0: 'prob_churn', 'index': 'ID'}, inplace=True) 
prob_combined

,ID,prob_churn
0,750,0.413712
1,650,0.541247
2,1719,0.389247
3,159,0.457714
4,3139,0.536754
...,...,...
5579,3330,0.408353
5580,70,0.310281
5581,132,0.448121
5582,2014,0.386264


In [409]:
# then append the actual churned or not churned y values
y_actual = y_train.append(y_test)
actual_combined = y_actual.reset_index()
actual_combined.rename(columns={0: 'actual_churn', 'index': 'ID'}, inplace=True) 
actual_combined

,ID,churned
0,1159,0
1,5404,0
2,579,0
3,1302,0
4,69,0
...,...,...
5579,2273,0
5580,1683,0
5581,2845,0
5582,2155,0


In [420]:
# combine the above two steps
ks_combined = pd.merge(prob_combined, actual_combined, how='left', on = 'ID')
ks_combined = ks_combined.sort_values('prob_churn',ascending=False)
ks_combined

,ID,prob_churn,churned
2187,326,0.864837,0
3795,3683,0.864837,1
3395,1680,0.857618,0
3433,5303,0.857618,0
593,4009,0.826561,0
...,...,...,...
128,5559,0.150066,0
1471,86,0.146273,0
728,4215,0.097782,0
1127,230,0.088677,0


In [422]:
# equally cut the data into 10 deciles 
lst = np.array_split(ks_combined, 10)

In [453]:
# create an empty dataframe for KS
df_ks = pd.DataFrame(columns=['Decile','Min_prob','Max_prob','Events', 'Non_events','Event_rate', 'Non_event_rate', 'cum_event_rate', 'cum_non_event_rate'])

# fill data to the empty KS df by looping thru the data
for i in range(0, len(lst)):
    decile = i+1
    min_prob = lst[i]['prob_churn'].min()
    max_prob = lst[i]['prob_churn'].max()
    events = lst[i][lst[i]['churned']==1].shape[0]
    nonevents = lst[i][lst[i]['churned']==0].shape[0]
    df_ks = df_ks.append({'Decile' : str(decile) , 'Min_prob' :min_prob, 'Max_prob' : max_prob,
                          'Events': events, 'Non_events': nonevents}, 
                            ignore_index = True)
    
df_ks

,Decile,Min_prob,Max_prob,Events,Non_events,Event_rate,Non_event_rate,cum_event_rate,cum_non_event_rate
0,1,0.671458,0.864837,57,502,NaN,NaN,NaN,NaN
1,2,0.582938,0.670601,47,512,NaN,NaN,NaN,NaN
2,3,0.544903,0.580818,29,530,NaN,NaN,NaN,NaN
3,4,0.497843,0.544903,27,532,NaN,NaN,NaN,NaN
4,5,0.448121,0.497843,27,531,NaN,NaN,NaN,NaN
5,6,0.408353,0.448121,28,530,NaN,NaN,NaN,NaN
6,7,0.373690,0.408353,15,543,NaN,NaN,NaN,NaN
7,8,0.335280,0.373690,19,539,NaN,NaN,NaN,NaN
8,9,0.297503,0.335280,13,545,NaN,NaN,NaN,NaN
9,10,0.079401,0.297503,17,541,NaN,NaN,NaN,NaN


In [462]:
# calculate event/non-event rates, cumulative event/non-event rates and KS score
df_ks['Event_rate'] = df_ks['Events'] / df_ks['Events'].sum() * 100 
df_ks['Non_event_rate'] = df_ks['Non_events'] / df_ks['Non_events'].sum() * 100
df_ks['cum_event_rate'] = df_ks['Event_rate'].cumsum()
df_ks['cum_non_event_rate'] = df_ks['Non_event_rate'].cumsum()
df_ks['KS'] = df_ks['cum_event_rate'] - df_ks['cum_non_event_rate'] 

In [463]:
df_ks

,Decile,Min_prob,Max_prob,Events,Non_events,Event_rate,Non_event_rate,cum_event_rate,cum_non_event_rate,KS
0,1,0.671458,0.864837,57,502,20.430108,9.462771,20.430108,9.462771,10.967337
1,2,0.582938,0.670601,47,512,16.845878,9.651272,37.275986,19.114043,18.161942
2,3,0.544903,0.580818,29,530,10.394265,9.990575,47.670251,29.104618,18.565633
3,4,0.497843,0.544903,27,532,9.677419,10.028275,57.34767,39.132893,18.214777
4,5,0.448121,0.497843,27,531,9.677419,10.009425,67.02509,49.142319,17.882771
5,6,0.408353,0.448121,28,530,10.035842,9.990575,77.060932,59.132893,17.928038
6,7,0.373690,0.408353,15,543,5.376344,10.235627,82.437276,69.36852,13.068756
7,8,0.335280,0.373690,19,539,6.810036,10.160226,89.247312,79.528746,9.718565
8,9,0.297503,0.335280,13,545,4.659498,10.273327,93.90681,89.802074,4.104737
9,10,0.079401,0.297503,17,541,6.09319,10.197926,100.0,100.0,0.0


# Questions B: 

Constructed Logistic regression to predict which user will churn, testing set has an AUC of 0.60298 while training set has an AUC of 0.64683. KS table indicates that if we were to implement the model, we would be applying the cut on 3rd decile which treat users with min probability of churn greater than 0.545 as churned. classification output is below:

In [473]:
feature_data_final = feature_data.reset_index()
feature_data_.rename(columns={'index': 'ID'}, inplace=True) 
feature_data_ = pd.merge(feature_data_[['ID','user_id']], ks_combined, how='left', on='ID')
feature_data_.drop(columns=['ID','churned'], inplace=True)
feature_data_

,user_id,prob_churn
0,895044c23edc821881e87da749c01034,0.520023
1,458b1d95441ced242949deefe8e4b638,0.573098
2,c7936f653d293479e034865db9bb932f,0.678044
3,b255d4bd6c9ba194d3a350b3e76c6393,0.515020
4,4a168225e89375b8de605cbc0977ae91,0.576476
...,...,...
5579,03880c726d8a4e5db006afe4119ad974,0.428438
5580,ae8315109657f44852b24c6bca4decd6,0.544903
5581,f29c174989f9737058fe808fcf264135,0.769142
5582,24843497d1de88b2e7233f694436cb3a,0.536993


# Question C
Based on the model result summary output and information value, variables risk_tolerance, platform, time_spent, instrument_type_first_traded, first_deposite_amount and time_horizon all display statistical significance (with p value less than 0.05).